In [28]:
from keras.models import load_model
from keras.layers import Input,LSTM,Dense
from keras import Model
from keras.models import load_model
from gensim.models import Word2Vec
import numpy as np
import editdistance

target_classes = ['check_all_classes' ,'check_for_reminders', 'check_assignment_due', 'check_when_class_is' , 'check_what_lecture' , 'add_reminder','where_class']

inp = Input(shape=[None,20])
lstm=LSTM(16)(inp)
out = Dense(7,activation='softmax')(lstm)
model_inf = Model(inputs = inp,outputs=out)
model_inf.load_weights('keras_model_weights.h5')

'''def intt_model(data):
    inp = Input(shape=[None,20])
    lstm=LSTM(16)(inp)
    out = Dense(7,activation='softmax')(lstm)
    model_inf = Model(inputs = inp,outputs=out)
    model_inf.load_weights('keras_model_weights.h5')
    return model_inf.predict(data)'''
    
w2v = Word2Vec.load('w2v_model_for_intent_classification.model')

def intent_classifier(sent):
    
    org_sen=sent
    
    def unk_words(unk):
        mini = 10
        for i in range(len(w2v.wv.vocab)):
            dist = editdistance.eval(unk,list(w2v.wv.vocab)[i])
            if list(w2v.wv.vocab)[i][0] == unk[0]:
                if dist < mini:
                    mini = dist
                    unk_closest = list(w2v.wv.vocab)[i]
        try:
            return unk_closest
        except:
            return "bad"


    sent = sent.split(" ")
    a = []
    for i in range(len(sent)):
        try:
            a.append(w2v.wv[sent[i]])
        except KeyError:
            sent[i] = unk_words(sent[i])
            if sent[i] == 'bad':
                return 'bad'
            a.append(w2v.wv[sent[i]])
    for i in range(10 - len(a)):
        a.append(w2v.wv['pad_word'])
    a = np.array(a)
    a = a.reshape([1,len(a),20])
    
    #return target_classes[np.argmax(intt_model(a)[0])],org_sen
    return target_classes[np.argmax(model_inf.predict(a)[0])],org_sen


In [45]:
import time
st=time.time()
intent_classifier("what is my first class today")
end=time.time()
end-st

0.004541873931884766

In [46]:
st=time.time()
intent_classifier("what is my first class today")
end=time.time()
end-st

0.008810997009277344